In [1]:
# Import necessary libraries
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import numpy as np

In [2]:
# Load the csv file as dataframe.
df = pd.read_csv('./data/face_cosmetics_products_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product_ID        36 non-null     int64 
 1   Product_Category  36 non-null     object
 2   Product_Type      36 non-null     object
 3   Brand_Name        36 non-null     object
 4   Product_Name      36 non-null     object
 5   Product_Details   36 non-null     object
 6   Benefits          36 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.1+ KB


In [4]:
# Configure the baseline configuration of the OpenAI library.
openai.api_type = "azure"
openai.api_base = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"
engine = "PLEASE_ENTER_YOUR_OWNED_AOAI_TEXT_EMBEDDING_MODEL_NAME"

In [5]:
# Perform word embedding for each product name in the dataframe in vector form.
embeddings = [get_embedding(text, engine=engine) for text in df['Benefits'].values]

In [6]:
# Calculate the cosine similarity between each product name.
similarity_matrix = np.zeros((len(embeddings), len(embeddings)))
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        if i == j:
            similarity_matrix[i,j] = 1.0
        elif i < j:
            similarity_matrix[i,j] = cosine_similarity(embeddings[i], embeddings[j])
        else:
            similarity_matrix[i,j] = similarity_matrix[j,i]

In [7]:
# Save the similarity matrix to a CSV file.
np.savetxt('./data/similarity_matrix_fc_b.csv', similarity_matrix, delimiter=',')

In [8]:
# Reload the similarity matrix from the CSV file.
# similarity_matrix = np.loadtxt('./data/similarity_matrix_fc_b.csv', delimiter=',')

In [9]:
# Create a list of Product_Name.
p_name = pd.Series(df['Product_Name'])
p_name

0     FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUND...
1                MIRACLE PURE SKIN IMPROVING FOUNDATION
2                         FACEFINITY COMPACT FOUNDATION
3                              MIRACLE TOUCH FOUNDATION
4                  MAX FACTOR CRÈME PUFF POWDER COMPACT
5                 FACEFINITY ALL DAY FLAWLESS CONCEALER
6                                MIRACLE PURE CONCEALER
7                                RADIANT LIFT CONCEALER
8                                 MASTERTOUCH CONCEALER
9                            MIRACLE CONTOURING PALETTE
10                                     MIRACLE GLOW DUO
11                             MIRACLE PURE CREAM BLUSH
12                           MIRACLE TOUCH CREAMY BLUSH
13       MIRACLE PURE VITAMIN C SKIN-ILLUMINATING SERUM
14                           BROW SHAPER EYEBROW PENCIL
15                                     BROW REVIVAL GEL
16                        BROWFINITY LONGWEAR BROW TINT
17                                       EYEBROW

In [10]:
# Create function for Product Recommendations.
def product_recommendations(Product_Name, cos_sim = similarity_matrix):
    recommended_product = []
    idx = p_name[p_name == Product_Name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_product.append(list(df['Product_Name'])[i])
    return recommended_product

In [11]:
# Perform Product Recommendations. It is based on Product Name to figure out the similarity.
product_recommendations('MIRACLE PURE SKIN IMPROVING FOUNDATION')

['MIRACLE PURE CONCEALER',
 'FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUNDATION',
 'FACEFINITY ALL DAY FLAWLESS CONCEALER',
 'RADIANT LIFT CONCEALER',
 'MIRACLE PURE VITAMIN C SKIN-ILLUMINATING SERUM',
 'MIRACLE PURE CREAM BLUSH',
 'MIRACLE TOUCH FOUNDATION',
 'MAX FACTOR CRÈME PUFF POWDER COMPACT',
 'MASTERTOUCH CONCEALER',
 'MASTERPIECE MONO EYESHADOW']

In [12]:
df_product_sim_matrix = pd.DataFrame(similarity_matrix, index = p_name, columns = p_name)
df_product_sim_matrix

Product_Name,FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUNDATION,MIRACLE PURE SKIN IMPROVING FOUNDATION,FACEFINITY COMPACT FOUNDATION,MIRACLE TOUCH FOUNDATION,MAX FACTOR CRÈME PUFF POWDER COMPACT,FACEFINITY ALL DAY FLAWLESS CONCEALER,MIRACLE PURE CONCEALER,RADIANT LIFT CONCEALER,MASTERTOUCH CONCEALER,MIRACLE CONTOURING PALETTE,...,FALSE LASH EPIC MASCARA,LASH REVIVAL EYELASH PRIMER,VOLUME INFUSION MASCARA,FALSE LASH EFFECT XXL MASCARA,VELVET MATTES LIPSTICK,COLOUR ELIXIR LIPSTICK,PRIYANKA COLOUR ELIXIR LIPSTICK,LIPFINITY LIQUID LIPSTICK,COLOUR ELIXIR LIP LINER,HONEY LACQUER
Product_Name,,,,,,,,,,,,,,,,,,,,,
FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUNDATION,1.000000,0.901918,0.869561,0.852081,0.860417,0.880974,0.881204,0.847455,0.857883,0.848702,...,0.804970,0.809329,0.810822,0.825452,0.834845,0.797393,0.818057,0.833248,0.794539,0.804878
MIRACLE PURE SKIN IMPROVING FOUNDATION,0.901918,1.000000,0.836826,0.865842,0.865671,0.891701,0.931275,0.876363,0.853708,0.839944,...,0.789119,0.807061,0.822360,0.802669,0.825002,0.809983,0.818797,0.808935,0.771210,0.778913
FACEFINITY COMPACT FOUNDATION,0.869561,0.836826,1.000000,0.817349,0.868777,0.834645,0.829735,0.838439,0.836184,0.833751,...,0.824512,0.801469,0.844392,0.804149,0.866582,0.829998,0.809840,0.861414,0.814608,0.836862
MIRACLE TOUCH FOUNDATION,0.852081,0.865842,0.817349,1.000000,0.846443,0.838397,0.853779,0.838068,0.866898,0.850122,...,0.802595,0.781835,0.826523,0.807284,0.821826,0.804620,0.793958,0.796635,0.784065,0.790856
MAX FACTOR CRÈME PUFF POWDER COMPACT,0.860417,0.865671,0.868777,0.846443,1.000000,0.873776,0.876807,0.878701,0.871707,0.879015,...,0.830187,0.816527,0.841076,0.819855,0.866829,0.818795,0.822035,0.839751,0.814166,0.835767
FACEFINITY ALL DAY FLAWLESS CONCEALER,0.880974,0.891701,0.834645,0.838397,0.873776,1.000000,0.924816,0.902835,0.894870,0.854656,...,0.808887,0.816748,0.808390,0.828781,0.846473,0.812494,0.843609,0.842865,0.805669,0.803342
MIRACLE PURE CONCEALER,0.881204,0.931275,0.829735,0.853779,0.876807,0.924816,1.000000,0.899308,0.871283,0.844214,...,0.799798,0.812857,0.824986,0.823323,0.836492,0.819381,0.831941,0.822540,0.788644,0.798433
RADIANT LIFT CONCEALER,0.847455,0.876363,0.838439,0.838068,0.878701,0.902835,0.899308,1.000000,0.900711,0.864070,...,0.840188,0.833136,0.862984,0.816498,0.853254,0.824191,0.807127,0.825035,0.823697,0.816142
MASTERTOUCH CONCEALER,0.857883,0.853708,0.836184,0.866898,0.871707,0.894870,0.871283,0.900711,1.000000,0.881131,...,0.847799,0.807842,0.831560,0.828118,0.859116,0.802296,0.800936,0.820654,0.819303,0.814973
